# co

> Covariance Matrix and Coherence Matrix Estimation

In [ ]:
#| default_exp co

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
# import for test
import numpy as np
import itertools

In [ ]:
#| export
import cupy as cp
import math

In [ ]:
#| export
# this function could be faster if the reducing is done 2 by 2
_co_mat_kernel = cp.RawKernel(r'''
#include <cuComplex.h>

extern "C" __global__
void co_mat(const cuFloatComplex* rslc, const bool* is_shp, const int nlines, const int width, const int nimages, const int az_half_win,const int r_half_win, cuFloatComplex* cov, cuFloatComplex* coh) {
    int tid = blockDim.x * blockIdx.x + threadIdx.x;
    if (tid >= nlines*width) return;
    int az_win = 2*az_half_win+1;
    int r_win = 2*r_half_win+1;
    int win = az_win*r_win;
    
    int ref_az = tid/width;
    int ref_r = tid -ref_az*width;

    int sec_az, sec_r;

    int i,j; // index of each coherence matrix
    int k,l; // index of search window
    cuFloatComplex _cov; // covariance
    float _amp2_i; // sum of amplitude square for image i
    float _amp2_j; // sum of amplitude aquare for image j
    int rslc_inx_i, rslc_inx_j;

    for (i = 0; i < nimages; i++) {
        for (j = 0; j < nimages; j++) {
            _cov = make_cuFloatComplex(0.0, 0.0);
            _amp2_i = 0.0;
            _amp2_j = 0.0;
            for (k = 0; k < az_win; k++) {
                for (l = 0; l < r_win; l++) {
                    sec_az = ref_az-az_half_win+k;
                    sec_r = ref_r-r_half_win+l;
                    if (is_shp[tid*win+k*r_win+l] && sec_az >= 0 && sec_az < nlines && sec_r >= 0 && sec_r < width) {
                        rslc_inx_i = (sec_az*width+sec_r)*nimages+i;
                        rslc_inx_j = (sec_az*width+sec_r)*nimages+j;
                        _amp2_i += powf(cuCrealf(rslc[rslc_inx_i]),2)+powf(cuCimagf(rslc[rslc_inx_i]),2);
                        _amp2_j += powf(cuCrealf(rslc[rslc_inx_j]),2)+powf(cuCimagf(rslc[rslc_inx_j]),2);
                        _cov = cuCaddf(_cov, cuCmulf(rslc[rslc_inx_i], cuConjf(rslc[rslc_inx_j])));
                        //if ( tid == 0 && i==1 && j ==1 ) printf("%f\n",cuCrealf(_cov));
                    }
                }
            }
            cov[(tid*nimages+i)*nimages+j] = _cov;
            _amp2_i = sqrt(_amp2_i*_amp2_j);
            coh[(tid*nimages+i)*nimages+j] = make_cuFloatComplex(cuCrealf(_cov)/_amp2_i, cuCimagf(_cov)/_amp2_i);
            //if ( tid == 0 && i==1) printf("%f\n",cuCrealf(_cov));
        }
    }
}
''', 'co_mat')

In [ ]:
#| export
def co_mat(rslc:cp.ndarray, # rslc stack, dtype: cupy.complex64
            is_shp:cp.ndarray, # shp bool, dtype: cupy.bool
            block_size:int=128, # the CUDA block size, it only affects the calculation speed
            )-> tuple: # the covariance and coherence matrix
    nlines, width, nimages = rslc.shape
    az_win, r_win = is_shp.shape[-2:]
    az_half_win = (az_win-1)//2
    r_half_win = (r_win-1)//2

    cov = cp.zeros((nlines,width,nimages,nimages),dtype=cp.complex64)
    coh = cp.empty((nlines,width,nimages,nimages),dtype=cp.complex64)

    grid_size = math.ceil(nlines*width/block_size)
    _co_mat_kernel((grid_size,),(block_size,),
                    (rslc, is_shp, cp.int32(nlines),cp.int32(width),cp.int32(nimages),
                    cp.int32(az_half_win),cp.int32(r_half_win),cov,coh))
    return cov,coh

The `rslc` is a three dimentional cupy `ndarray`. The `dtype` should be `cupy.complex64`. From outerest to innerest, the three dimentions are azimuth, range and image.
`is_shp` is a four dimentional cupy `ndarray`. From outerest ot innerest, they are azimuth, range, secondary pixel relative azimuth, secondary pixel relative range.

Here is an example:

In [ ]:
import cupy as cp
rslc = cp.load('../../data/rslc.npy')
rslc = rslc[:5,:10,:5].copy()
rslc.shape

(5, 10, 5)

`rslc` is a stack of 5 rslc images. Each of the image has 5 pixel in azimuth dimention and 10 pixels in range dimention.
Apply ks test on it:

In [ ]:
from decorrelation.shp import ks_test
rmli = cp.abs(rslc)**2
sorted_rmli = cp.sort(rmli,axis=-1)
dist, p = ks_test(sorted_rmli,az_half_win=1,r_half_win=1)

Seclect SHP based on p value:

In [ ]:
is_shp = (p < 0.005) & (p >= 0.0)

Estimate the covarience and coherence matrix: 

In [ ]:
cov,coh = co_mat(rslc,is_shp)
cov.shape, coh.shape

((5, 10, 5, 5), (5, 10, 5, 5))

Both `cov` and `coh` are complex data. The shape shows each covarience or coherence matrix is 5 by 5 since there are 5 images.
And `cov` and `coh` are matrix for all 5*10 pixels.

In [ ]:
#| hide
# test

# az, r, image, image
half_az_win = is_shp.shape[2]//2;
half_r_win = is_shp.shape[3]//2;
for i, j, k, l in itertools.product(range(rslc.shape[0]),range(rslc.shape[1]),range(rslc.shape[2]),range(rslc.shape[2])):
    _cov = 0.0+0.0j
    _amp2_k = 0.0
    _amp2_l = 0.0
    # shp_az, shp_r
    for m, n in itertools.product(range(is_shp.shape[2]),range(is_shp.shape[3])):
        if is_shp[i,j,m,n]:
            _cov += rslc[i+m-half_az_win,j+n-half_r_win,k]*rslc[i+m-half_az_win,j+n-half_r_win,l].conj()
            _amp2_k += abs(rslc[i+m-half_az_win,j+n-half_r_win,k])**2
            _amp2_l += abs(rslc[i+m-half_az_win,j+n-half_r_win,l])**2
    assert abs(_cov-cov[i,j,k,l])<1.0e-7
    assert abs(_cov/math.sqrt(_amp2_k*_amp2_l) - coh[i,j,k,l]) < 1.0e-6

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()